In [1]:
import os
from time import time as realtime
from pytools_uibcdf.Time import formatted_elapsed_time, formatted_local_time

import shutil
import molmodmt as m3t
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit

from openmmtools.integrators import LangevinIntegrator
import pickle

from mdtraj.reporters import HDF5Reporter

/home/liliana/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab/lib/python3.7/site-packages/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


In [2]:
#os.remove('./logfile.txt')

#### Files Produced

##### In current dir

In [3]:
log_file = 'logfile.txt'

##### In temporal dir

In [4]:
trajectory_file = "trajectory.h5"
checkpoint_file = "checkpoint.chk"
final_state_file = "final_state.xml"
final_checkpoint_file = "final_checkpoint.chk"
final_pdb_file = "final_positions.pdb"


#### Log

In [5]:
log = open(log_file,'w')

In [6]:
start_realtime = realtime()
log.write("\n")
log.write("Start: "+formatted_local_time()+"\n")
log.write("\n")

1

#### Loading PDB

In [7]:
modelo_HIF_init= m3t.convert('HIF1_equilibrado_NPT.pdb', 'openmm.Modeller')

#### System

In [8]:
topology = modelo_HIF_init.topology
forcefield = app.ForceField('amber99sbildn.xml','tip3p.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*unit.nanometers, 
                        constraints=app.HBonds,rigidWater=True, ewaldErrorTolerance=0.0005)

#### Thermodynamic State

In [9]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0*unit.kelvin
pressure    = 1.0*unit.atmosphere

#### Integrator

In [10]:
friction   = 1.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

#### Barostat

In [11]:
barostat_interval = 25
barostat = mm.MonteCarloBarostat(pressure, temperature, barostat_interval)
system.addForce(barostat)

5

#### Platform

In [12]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

#### Simulation

In [13]:
simulation = app.Simulation(topology, system, integrator, platform, properties)

#### Initial Conditions

In [14]:
restart_pkl = open("restart.pkl","rb")

_, init_positions, init_velocities, init_box_vectors = pickle.load(restart_pkl)
init_positions = init_positions * unit.nanometers
init_velocities = init_velocities * unit.nanometers*unit.picoseconds
init_box_vectors = init_box_vectors * unit.nanometers

restart_pkl.close()

simulation.context.setPositions(init_positions)
simulation.context.setVelocities(init_velocities)

simulation.context.setPeriodicBoxVectors(*init_box_vectors)
#simulation.context.setPeriodicBoxVectors(init_box_vectors[0],init_box_vectors[1],init_box_vectors[2])


#### Iterations Parameters

In [15]:
steps_simulation = 5000 # 10 ps
steps_interval_saving = 1000 # x ps 
steps_interval_verbose = 200 # x ns
steps_interval_checkpoint = 1000 # 1n

time_simulation = (steps_simulation*step_size).in_units_of(unit.nanoseconds)
time_saving = (steps_interval_saving*step_size).in_units_of(unit.picoseconds)
time_verbose = (steps_interval_verbose*step_size).in_units_of(unit.picoseconds)
time_checkpoint = (steps_interval_checkpoint*step_size).in_units_of(unit.picoseconds)

log.write("\n")
log.write("Step size: {}\n".format(step_size))
log.write("Simulation time: {} ({} steps)\n".format(time_simulation , steps_simulation))
log.write("Saving time: {} ({} steps)\n".format(time_saving , steps_interval_saving))
log.write("Verbose time: {} ({} steps)\n".format(time_verbose , steps_interval_verbose))
log.write("Checkpoint time: {} ({} steps)\n".format(time_checkpoint , steps_interval_checkpoint))
log.write("\n")

1

#### Reporters

#### Logfile

In [16]:
simulation.reporters.append(app.StateDataReporter(log, reportInterval=steps_interval_verbose,
                                                  progress=True, speed=True, step=True, time=True,
                                                  potentialEnergy=True, temperature=True,
                                                  volume=True, totalSteps=steps_simulation,
                                                  separator=", "))

#### Observables

In [17]:
simulation.reporters.append(HDF5Reporter(trajectory_file, reportInterval=steps_interval_saving,
                                         coordinates=True, time=True, cell=True,
                                         potentialEnergy=True, kineticEnergy=True,
                                         temperature=True))

#### Checkpoints

In [18]:
simulation.reporters.append(app.CheckpointReporter(checkpoint_file, 
                                                   steps_interval_checkpoint))

#### Running Simulation

In [19]:
start_simulation_realtime = realtime()

simulation.step(steps_simulation)

end_simulation_realtime = realtime()

#### Saving Finnal State

In [20]:
simulation.saveState(final_state_file)
simulation.saveCheckpoint(final_checkpoint_file)
m3t.convert(simulation,final_pdb_file)

#### Removing partial checkpoints

In [21]:
os.remove(checkpoint_file)

#### Closing all reporters but Log

In [22]:
simulation.reporters[1].close()

#### Summary

In [23]:
end_realtime = realtime()
preparation_elapsed_realtime = (start_simulation_realtime - start_realtime)*unit.seconds
simulation_elapsed_realtime = (end_simulation_realtime - start_simulation_realtime)*unit.seconds
total_elapsed_realtime = (end_realtime - start_realtime)*unit.seconds

performance = 24 * (time_simulation/unit.nanoseconds) / (simulation_elapsed_realtime/unit.hours)

log.write("\n")
log.write("End: "+formatted_local_time()+"\n")
log.write("\n")
log.write("****SUMMARY****\n")
log.write("\n")
log.write("Total time: "+formatted_elapsed_time(total_elapsed_realtime/unit.seconds)+"\n")
log.write("Preparation time: "+formatted_elapsed_time(preparation_elapsed_realtime/unit.seconds)+"\n")
log.write("Simulation time: "+formatted_elapsed_time(simulation_elapsed_realtime/unit.seconds)+"\n")
log.write("\n")
log.write("Simulation Performance: {:.3f} ns/day".format(performance)+"\n")
log.write("\n")

1

#### Closing Log

In [24]:
log.close()